In [1]:
import json
import random
import copy

import time 
import numpy as np
import matplotlib.pyplot as plt
import cv2
import h5py
import os
import torch

from tqdm import tqdm
from PIL import Image
from sklearn.preprocessing import StandardScaler

import webdataset as wds
import sys

from utils import seed_everything

%load_ext autoreload
%autoreload 2

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:",device)

device: cuda


In [2]:
# CLIP
import clip
from torchvision import transforms
from models import *
clip_extractor = Clipper("ViT-L/14", device=device, hidden_state=True, norm_embs=False)
clip_extractor_last = Clipper("ViT-L/14", device=device, hidden_state=False, norm_embs=False)

openclip_extractor = OpenClipper('ViT-H-14', device=device, hidden_state=False, norm_embs=False)
openclip_extractor_last = OpenClipper('ViT-H-14', device=device, hidden_state=True, norm_embs=False)

# ImageBind
import sys
sys.path.insert(0, "/fsx/proj-medarc/fmri/ImageBind")
sys.path.insert(0, "/fsx/proj-medarc/fmri/ImageBind/models")
import data
import imagebind_model
from imagebind_model import ModalityType

imagebind = imagebind_model.imagebind_huge(pretrained=True)
imagebind.eval().requires_grad_(False)
imagebind.to(device)

imagebind_transform = transforms.Compose(
    [transforms.Resize(224, interpolation=transforms.InterpolationMode.BICUBIC),
    transforms.CenterCrop(224),
    transforms.Normalize(
        mean=(0.48145466, 0.4578275, 0.40821073),
        std=(0.26862954, 0.26130258, 0.27577711))])

imagebind_hidden = copy.deepcopy(imagebind)
imagebind_hidden.modality_heads.vision = nn.Identity()
imagebind_hidden.modality_postprocessors.vision = nn.Identity()
imagebind_hidden.eval().requires_grad_(False)
imagebind_hidden.to(device)

# FCN segmentation model
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
fcn_weights = FCN_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1
fcn_model = create_feature_extractor(fcn_resnet50(weights=fcn_weights), return_nodes=["backbone.maxpool"]).to(device)
fcn_model.eval().requires_grad_(False)
pass

ViT-L/14 cuda
ViT-L/14 cuda
ViT-H-14 cuda
ViT-H-14 cuda
THIS IS NOT WORKING CURRENTLY!


/fsx/home-paulscotti/miniconda3/envs/medical-v1/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/fsx/home-paulscotti/miniconda3/envs/medical-v1/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


Note that the data have already been session-wise z-score normalized and averaged across same-image repetitions! In fsaverage surface space ("challenge" space), which may be equivalent to nsdgeneral.

In [3]:
for sub in [1]: # [1,2,3,4,5,6,7,8]
    #1 2 5 7 are the subjects who completed all trials
    subj=f'subj0{sub}'
    print(subj)
    
    seed_everything(0)

    samples_per_shard = 300 # samples per tar

    base_path = f"/fsx/proj-medarc/fmri/natural-scenes-dataset/algonauts_data/dataset/{subj}"
    wds_path = base_path + "/webdataset"
    os.makedirs(wds_path,exist_ok=True)
    print("base_path:",base_path)

    # number of training samples
    files = os.listdir(base_path+"/training_split/training_images")
    train_png_files = [file for file in files if file.endswith('.png')]
    num_train = len(train_png_files)
    print("# training: ", num_train)
    train_imgs = np.array([np.array(Image.open(base_path+"/training_split/training_images/"+p)) for p in train_png_files])

    lh_fmri = np.load(base_path+"/training_split/training_fmri/lh_training_fmri.npy")
    rh_fmri = np.load(base_path+"/training_split/training_fmri/rh_training_fmri.npy")
    train_fmri = np.hstack((lh_fmri,rh_fmri))
    
    # Shuffle both matrices along the first axis
    permutation = np.random.permutation(train_imgs.shape[0])
    train_imgs = train_imgs[permutation]
    train_fmri = StandardScaler().fit_transform(train_fmri[permutation])

    # number of val samples
    files = os.listdir(base_path+"/test_split/test_images")
    test_png_files = [file for file in files if file.endswith('.png')]
    num_test = len(test_png_files)
    print("# test: ", num_test)
    test_imgs = np.array([np.array(Image.open(base_path+"/test_split/test_images/"+p)) for p in test_png_files])

    # train & val
    abs_cnt = -1
    for idx,i in enumerate(tqdm(range(0, num_train, samples_per_shard))):
        cur_imgs = np.moveaxis(train_imgs[i:i+samples_per_shard],-1,1)
        cur_imgs_tensor = torch.Tensor(cur_imgs).to(device)
        cur_samps = train_fmri[i:i+samples_per_shard]
        with torch.no_grad():
            clip_emb_hidden = clip_extractor.embed_image(cur_imgs_tensor)
            clip_emb_last = clip_extractor_last.embed_image(cur_imgs_tensor)

            ib_inputs = {ModalityType.VISION: imagebind_transform(cur_imgs_tensor)}

            ib_emb_last = imagebind(ib_inputs)['vision']
            ib_emb_hidden = imagebind_hidden(ib_inputs)['vision']

            seg_emb = fcn_model(cur_imgs_tensor)['backbone.maxpool']

        if idx==0: # val
            print("vert",cur_samps.shape)
            print("clip_vitl_hidden",clip_emb_hidden.shape)
            print("clip_vitl_final",clip_emb_last.shape)
            print("imagebind_hidden",ib_emb_hidden.shape)
            print("imagebind_final",ib_emb_last.shape)
            print("fcn_maxpool",seg_emb.shape)
            print("image",cur_imgs.shape)
            
            os.makedirs(wds_path+"/val",exist_ok=True)
            sink = wds.TarWriter(wds_path+f"/val/{subj}_{idx}.tar")
        else:
            os.makedirs(wds_path+"/train",exist_ok=True)
            sink = wds.TarWriter(wds_path+f"/train/{subj}_{idx-1}.tar")
            
        for ii in range(len(cur_samps)):
            abs_cnt += 1
            sink.write({
                "__key__": "sample%09d" % abs_cnt,
                "vert.npy": cur_samps[ii],
                "trial.npy": np.array([i+ii]),
                "clip_vitl_hidden.npy": clip_emb_hidden[ii].detach().cpu().numpy(),
                "clip_vitl_final.npy": clip_emb_last[ii].detach().cpu().numpy(),
                "imagebind_hidden.npy": ib_emb_hidden[ii].detach().cpu().numpy(),
                "imagebind_final.npy": ib_emb_last[ii].detach().cpu().numpy(),
                "fcn_maxpool.npy": seg_emb[ii].detach().cpu().numpy(),
                "image.npy": cur_imgs[ii],
            })
        sink.close()
        
    # test
    os.makedirs(wds_path+"/test",exist_ok=True)
    abs_cnt = -1
    for idx,i in enumerate(tqdm(range(0, num_test, samples_per_shard))):
        sink = wds.TarWriter(wds_path+f"/test/{subj}_{idx}.tar")

        cur_imgs = np.moveaxis(test_imgs[i:i+samples_per_shard],-1,1)
        cur_imgs_tensor = torch.Tensor(cur_imgs).to(device)
        with torch.no_grad():
            clip_emb_hidden = clip_extractor.embed_image(cur_imgs_tensor)
            clip_emb_last = clip_extractor_last.embed_image(cur_imgs_tensor)

            ib_inputs = {ModalityType.VISION: imagebind_transform(cur_imgs_tensor)}

            ib_emb_last = imagebind(ib_inputs)['vision']
            ib_emb_hidden = imagebind_hidden(ib_inputs)['vision']

            seg_emb = fcn_model(cur_imgs_tensor)['backbone.maxpool']
        
        for ii in range(len(cur_imgs)):
            abs_cnt += 1
            sink.write({
                "__key__": "sample%09d" % abs_cnt,
                "trial.npy": np.array([i+ii]),
                "clip_vitl_hidden.npy": clip_emb_hidden[ii].detach().cpu().numpy(),
                "clip_vitl_final.npy": clip_emb_last[ii].detach().cpu().numpy(),
                "imagebind_hidden.npy": ib_emb_hidden[ii].detach().cpu().numpy(),
                "imagebind_final.npy": ib_emb_last[ii].detach().cpu().numpy(),
                "fcn_maxpool.npy": seg_emb[ii].detach().cpu().numpy(),
                "image.npy": cur_imgs[ii],
            })
        sink.close()

    # So that we have the info, let's write this to a file
    with open(wds_path+f"/metadata_{subj}.json", "w") as f:    
        f.write(json.dumps({
            'train': num_train-samples_per_shard,
            'val': samples_per_shard,
            'test': num_test,
            'lh_samps': lh_fmri.shape[-1],
            'rh_samps': rh_fmri.shape[-1],
            'totals': num_train+num_test, 
        }, indent=4))

subj01
base_path: /fsx/proj-medarc/fmri/natural-scenes-dataset/algonauts_data/dataset/subj01
# training:  9841
# test:  159


  0%|          | 0/33 [00:00<?, ?it/s]

/fsx/home-paulscotti/miniconda3/envs/medical-v1/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


vert (300, 39548)
clip_vitl_hidden torch.Size([300, 257, 768])
clip_vitl_final torch.Size([300, 768])
imagebind_hidden torch.Size([300, 257, 1280])
imagebind_final torch.Size([300, 1024])
fcn_maxpool torch.Size([300, 64, 107, 107])
image (300, 3, 425, 425)


  0%|          | 0/1 [00:00<?, ?it/s]

# Upload to hf

In [37]:
from huggingface_hub import notebook_login, upload_file
notebook_login()

In [38]:
from tqdm import tqdm

for sub in [0]: #np.arange(8):
    subj=f'subj0{sub+1}'
    print(f"{subj}...")
    for tar in tqdm(range(0,32)):
        upload_file(
            path_or_fileobj=f"/fsx/proj-medarc/fmri/natural-scenes-dataset/algonauts_data/dataset/{subj}/webdataset/train/{subj}_{tar}.tar",
            path_in_repo=f"algonauts/train/{subj}_{tar}.tar",
            repo_id="pscotti/naturalscenesdataset",
            repo_type="dataset")

    tar = 0
    upload_file(path_or_fileobj=
                f"/fsx/proj-medarc/fmri/natural-scenes-dataset/algonauts_data/dataset/{subj}/webdataset/val/{subj}_{tar}.tar",
        path_in_repo=f"algonauts/val/{subj}_{tar}.tar",
        repo_id="pscotti/naturalscenesdataset",
        repo_type="dataset")

    upload_file(path_or_fileobj=
                f"/fsx/proj-medarc/fmri/natural-scenes-dataset/algonauts_data/dataset/{subj}/webdataset/test/{subj}_{tar}.tar",
        path_in_repo=f"algonauts/test/{subj}_{tar}.tar",
        repo_id="pscotti/naturalscenesdataset",
        repo_type="dataset")

    upload_file(
    path_or_fileobj=f"/fsx/proj-medarc/fmri/natural-scenes-dataset/algonauts_data/dataset/{subj}/webdataset/metadata_{subj}.json",
    path_in_repo=f"algonauts/metadata_{subj}.json",
    repo_id="pscotti/naturalscenesdataset",
    repo_type="dataset")

    print('done!')

subj01...


  0%|                                                    | 0/32 [00:00<?, ?it/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

subj01_0.tar:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

  3%|█▍                                          | 1/32 [00:34<17:37, 34.11s/it]

...

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

subj01_0.tar:   0%|          | 0.00/1.73G [00:00<?, ?B/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

subj01_0.tar:   0%|          | 0.00/890M [00:00<?, ?B/s]

done!
